# Decoder 
removed the argparse and hardcoded the filename

In [ ]:
import os
import math
import numpy as np
from jpeg_utils import *
from scipy import fftpack
from PIL import Image

In [ ]:

class JPEGFileReader:
    TABLE_SIZE_BITS = 16
    BLOCKS_COUNT_BITS = 32
    IMAGE_WIDTH_BITS = 16
    Q_strength_BITS = 8

    DC_CODE_LENGTH_BITS = 4
    CATEGORY_BITS = 4

    AC_CODE_LENGTH_BITS = 8
    RUN_LENGTH_BITS = 4
    SIZE_BITS = 4

    def __init__(self, filepath):
        self.__file = open(filepath, 'r')

    def read_int(self, size):
        if size == 0:
            return 0

        # the most significant bit indicates the sign of the number
        bin_num = self.__read_str(size)
        if bin_num[0] == '1':
            return self.__int2(bin_num)
        else:
            return self.__int2(binstr_flip(bin_num)) * -1

    def read_dc_table(self):
        table = dict()

        table_size = self.__read_uint(self.TABLE_SIZE_BITS)
        for _ in range(table_size):
            category = self.__read_uint(self.CATEGORY_BITS)
            code_length = self.__read_uint(self.DC_CODE_LENGTH_BITS)
            code = self.__read_str(code_length)
            table[code] = category
        return table

    def read_ac_table(self):
        table = dict()

        table_size = self.__read_uint(self.TABLE_SIZE_BITS)
        for _ in range(table_size):
            run_length = self.__read_uint(self.RUN_LENGTH_BITS)
            size = self.__read_uint(self.SIZE_BITS)
            code_length = self.__read_uint(self.AC_CODE_LENGTH_BITS)
            code = self.__read_str(code_length)
            table[code] = (run_length, size)
        return table

    def read_blocks_count(self):
        return self.__read_uint(self.BLOCKS_COUNT_BITS)
    
    def read_image_width(self):
        return self.__read_uint(self.IMAGE_WIDTH_BITS)
    
    def read_Q_strength(self):
        return self.__read_uint(self.Q_strength_BITS)
    
    def read_huffman_code(self, table):
        prefix = ''
        # TODO: break the loop if __read_char is not returing new char
        while prefix not in table:
            prefix += self.__read_char()
        return table[prefix]

    def __read_uint(self, size):
        if size <= 0:
            raise ValueError("size of unsigned int should be greater than 0")
        return self.__int2(self.__read_str(size))

    def __read_str(self, length):
        return self.__file.read(length)

    def __read_char(self):
        return self.__read_str(1)

    def __int2(self, bin_num):
        return int(bin_num, 2)


def read_image_file(filepath):
    reader = JPEGFileReader(filepath)

    tables = dict()
    for table_name in ['dc_y', 'ac_y', 'dc_c', 'ac_c']:
        if 'dc' in table_name:
            tables[table_name] = reader.read_dc_table()
        else:
            tables[table_name] = reader.read_ac_table()

    blocks_count = reader.read_blocks_count()
    image_width = reader.read_image_width()
    Q_strength = reader.read_Q_strength()

    dc = np.empty((blocks_count, 3), dtype=np.int32)
    ac = np.empty((blocks_count, 63, 3), dtype=np.int32)

    for block_index in range(blocks_count):
        for component in range(3):
            dc_table = tables['dc_y'] if component == 0 else tables['dc_c']
            ac_table = tables['ac_y'] if component == 0 else tables['ac_c']

            category = reader.read_huffman_code(dc_table)
            dc[block_index, component] = reader.read_int(category)

            cells_count = 0

            # TODO: try to make reading AC coefficients better
            while cells_count < 63:
                run_length, size = reader.read_huffman_code(ac_table)

                if (run_length, size) == (0, 0):
                    while cells_count < 63:
                        ac[block_index, cells_count, component] = 0
                        cells_count += 1
                else:
                    for i in range(run_length):
                        ac[block_index, cells_count, component] = 0
                        cells_count += 1
                    if size == 0:
                        ac[block_index, cells_count, component] = 0
                    else:
                        value = reader.read_int(size)
                        ac[block_index, cells_count, component] = value
                    cells_count += 1

    return dc, ac, tables, blocks_count, image_width, Q_strength


def zigzag_to_block(zigzag):
    # assuming that the width and the height of the block are equal
    rows = cols = int(math.sqrt(len(zigzag)))

    if rows * cols != len(zigzag):
        raise ValueError("length of zigzag should be a perfect square")

    block = np.empty((rows, cols), np.int32)

    for i, point in enumerate(zigzag_points(rows, cols)):
        block[point] = zigzag[i]

    return block


def dequantize(block, component, Q_strength):
    q = load_quantization_table(component)
    return block * q * Q_strength


def idct_2d(image):
    return fftpack.idct(fftpack.idct(image.T, norm='ortho').T, norm='ortho')



In [ ]:

def decode(filename):

    dc, ac, tables, blocks_count, image_width, Q_strength = read_image_file(filename)
    # assuming that the block is a 8x8 square
    print(f'Number of 8x8 blocks: {blocks_count}')
    block_side = 8

    # assuming that the image height and width are equal
    #image_side = int(math.sqrt(blocks_count)) * block_side
    #blocks_per_line = image_side // block_side
    
    blocks_per_line = image_width
    cols = image_width * block_side
    rows = blocks_count//image_width * block_side
    
    npmat = np.empty((rows, cols, 3))#, dtype=np.uint8)

    for block_index in range(blocks_count):
        i = block_index // blocks_per_line * block_side
        j = block_index % blocks_per_line * block_side
        
        for c in range(3):
            zigzag = [dc[block_index, c]] + list(ac[block_index, :, c])
            quant_matrix = zigzag_to_block(zigzag)
            dct_matrix = dequantize(quant_matrix, 'lum' if c == 0 else 'chrom', Q_strength)
            block = idct_2d(dct_matrix)
            npmat[i:i+8, j:j+8, c] = block + 128
    print(np.min(npmat),np.max(npmat))
    # The conversion to "uint8" create some artefacts
    image = Image.fromarray(npmat.astype(np.uint8), 'YCbCr')
    image = image.convert('RGB')
    #image.show()
    return image

In [ ]:
#filename = "UiTcompressed.jpg"
#uncompressed_file = 'UiTuncompressed.bmp'
filename = "coffeecompressed.jpg"
uncompressed_file = 'coffeeuncompressed.bmp'
#filename = "astronautcompressed.jpg"
#uncompressed_file = 'astronautuncompressed.bmp'


img = decode(filename)
img.save(uncompressed_file)
print(f"""Image in {filename} of size {os.stat(filename).st_size:,}
          uncompressed and saved as 
          {uncompressed_file} of size {os.stat(uncompressed_file).st_size:,}.""")
print("Warning, some artefacts appear due to the conversion of the iDCT values into uint8")

In [ ]:
img

In [ ]:
np.array(256).astype(np.uint8)